In [1]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("mini-contest/dataset.csv")

# Display the first few rows of the DataFrame
print(df.head())

   id                                          project_a  \
0   2  https://github.com/prettier-solidity/prettier-...   
1   3  https://github.com/prettier-solidity/prettier-...   
2   4  https://github.com/prettier-solidity/prettier-...   
3   5  https://github.com/prettier-solidity/prettier-...   
4   6  https://github.com/prettier-solidity/prettier-...   

                                    project_b  weight_a  weight_b  
0  https://github.com/nomicfoundation/hardhat  0.101669  0.898331  
1           https://github.com/consensys/teku  0.669446  0.330554  
2        https://github.com/ethereum/solidity  0.449022  0.550978  
3   https://github.com/ethereum/remix-project  0.498396  0.501604  
4     https://github.com/ethereum/go-ethereum  0.272503  0.727497  


In [4]:
from deepfunding import run_comparison
from tools.data_collection import fetch_repo_metrics

for i in range(len(df)):
    repo_a_url = df.iloc[i]["project_a"]
    repo_a = { "url": repo_a_url, **fetch_repo_metrics(repo_a_url) }
    repo_b_url = df.iloc[i]["project_b"]
    repo_b = { "url": repo_b_url, **fetch_repo_metrics(repo_b_url) }
    print(f"Repo A: {repo_a}")
    print(f"Repo B: {repo_b}")

    agent_result = run_comparison(repo_a, repo_b)
    print(f"Agent Result: {agent_result['weights']}")
    print(f"Human Result: {df.iloc[i]['weight_a']} {df.iloc[i]['weight_b']}")
    input("Press Enter to continue...")


Repo A: {'url': 'https://github.com/prettier-solidity/prettier-plugin-solidity', 'activeDeveloperCount6Months': 1, 'closedIssueCount6Months': 2, 'commitCount6Months': 15, 'contributorCount': 108, 'contributorCount6Months': 5, 'firstCommitDate': '2018-10-31 19:58:20', 'forkCount': '75', 'fulltimeDeveloperAverage6Months': 0, 'lastCommitDate': '2024-12-23 06:39:29', 'mergedPullRequestCount6Months': 23, 'newContributorCount6Months': 1, 'openedIssueCount6Months': 2, 'openedPullRequestCount6Months': 55, 'starCount': '737', 'total_funders_count': 12, 'total_funding_received_usd': 399520.38302470004, 'total_funding_received_usd_6_months': 0}
Repo B: {'url': 'https://github.com/nomicfoundation/hardhat', 'activeDeveloperCount6Months': 10, 'closedIssueCount6Months': 167, 'commitCount6Months': 265, 'contributorCount': 1656, 'contributorCount6Months': 155, 'firstCommitDate': '2022-03-16 11:55:37', 'forkCount': '1504', 'fulltimeDeveloperAverage6Months': 0, 'lastCommitDate': '2025-01-03 15:21:11', 'm

Error during comparison: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT


Raw event: {'validator': {'messages': [HumanMessage(content='{"validation": "The analyses provided by the project_analyzer, funding_strategist, and community_advocate cover various essential aspects, but there are areas that need further elaboration and justification:\\n\\n1. **Project Analyzer**: While the user adoption metrics are a significant improvement, the analysis lacks specific numerical data to illustrate the metrics. For example, the NPM downloads should be backed with actual numbers rather than generic statements like \'100 downloads per month\'. Additionally, it would be beneficial to include metrics on how frequently the plugin is mentioned in developer resources. Thus, the project_analyzer should revisit this analysis to include specific data points and examples.\\n\\n2. **Funding Strategist**: The funding analysis provides a strong justification for the funding received and community engagement metrics, but it lacks comparative analysis with the prettier-plugin-solidity

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT